## 전처리

In [1]:
import re
import pandas as pd

#특수 문자를 제거하고 연속된 공백을 하나로 줄인다.
def remove_escape(raw_text: str) -> str:
    pattern = r"\t|\n|\xa0"
    processed_text = re.sub(pattern, " ", raw_text)
    processed_text_stripped = " ".join(processed_text.split())
    return processed_text_stripped

def remove_hanja(text):
    # Unicode 범위를 사용하여 한자 제거
    return re.sub(r'[\u4e00-\u9fff]+', '', text)

def remove_hyperlink(raw_text: str) -> str:
    pattern = (
        r":*\s*\(*:*\s*https?://[\w\dㄱ-ㅎㅏ-ㅣ가-힣!@#$%^&*(),.?/:;\"'<>{}|+=~_-]+\s*\)*"
    )
    processed_text = re.sub(pattern, "", raw_text)
    return processed_text

#텍스트 시작 부분 헤더 제거
def remove_header(raw_text: str) -> str:
    header_pattern = "안녕하십니까. 대한법률구조공단 사이버상담을 이용해 주셔서 감사합니다."
    header_end_idx = re.search(header_pattern, raw_text)
    if header_end_idx != None:
        processed_text = raw_text[header_end_idx.end() :]
        return processed_text
    else:
        return raw_text
    
def remove_footer(text):
    # 제거할 문구를 정규 표현식으로 정의
    pattern = r'※ 주의 : 사례에 대한 답변은 법령이나 판례 등의 변경으로 내용이 바뀔 수 있으므로 구체적인 사안에 대해서는 반드시 대한법률구조공단 상담\(전화상담은 국번없이 ☎ 132\) 등을 통해 다시 한 번 확인하시기 바랍니다.'
    
    # 정규 표현식을 사용하여 해당 문구 제거
    cleaned_text = re.sub(pattern, '', text)
    
    # 앞뒤 공백 제거
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

def remove_info(text):
    # 작성자 정보 제거
    text = re.sub(r'작성자:\s*[\w\s]+', '', text)

    # 체류자격 미등록 제거
    text = re.sub(r'체류자격: 미등록', '',text)

    # 상담지원 단체 제거
    text = re.sub(r'상담지원단체:\s*[\w\s]+', '',text)
    
    # URL 제거
    text = re.sub(r'URL:\s*https?://\S+', '', text)
    
    # 마지막 줄바꿈 제거
    text = text.strip()
    
    return text


#특정 키워드가 포함된 문장 제거
def remove_page_word(raw_text: str) -> str:
    
    pattern = '사이버상담|사이버 상담|공단|방문|국번없이 132|132번'
    if re.findall(pattern, raw_text) == []:
        return raw_text
    
    split_text = raw_text.split('.')
    remove_text = [i for i in split_text if re.findall(pattern, i) == []]        

    return '.'.join(remove_text)

def remove_phone_number(raw_text: str) -> str:
    pattern = r'\b(\d{2,3}-\d{3,4}-\d{4}|\d{2}-\d{3}-\d{4})\b'
    processed_text = re.sub(pattern, "", raw_text)
    return processed_text

def remove_date_and_label(text):
    # '작성일:' 문구와 그 뒤의 날짜를 찾아 제거
    pattern = r'작성일:\s*\d{2,4}[-/년]\s*\d{1,2}[-/월]\s*\d{1,2}[일]?'
    text = re.sub(pattern, '', text)
    
    # 다른 형식의 날짜도 제거
    date_pattern = r'\d{2,4}[-/년]\s*\d{1,2}[-/월]\s*\d{1,2}[일]?'
    text = re.sub(date_pattern, '', text)

    # 날짜 형식 제거
    text = re.sub(r'\d{1,2}월\s?\d{1,2}일', '', text)
    text = re.sub(r'\d{2,4}[-.]\d{1,2}[-.]\d{1,2}', '', text)
    text = re.sub(r'\d{2,4}[-.]\d{1,2}[-.]\d{1,2}', '', text)
    text = re.sub(r'\d{2}\.\d{2}\.', '', text)  # OO.OO.
    
    return text.strip()


def preprocess(raw_text: str) -> str:
    preprocessed_text = raw_text
    preprocess_functions = [
        remove_hanja,
        remove_header,
        remove_footer,
        #remove_escape,
        remove_phone_number,
        #remove_page_word,
        remove_hyperlink,
        remove_info,
        #remove_link,
        remove_date_and_label,
        
    ]
    for preprocess_function in preprocess_functions:
        preprocessed_text = preprocess_function(preprocessed_text)
    return preprocessed_text

In [2]:
import json
import pandas as pd

# 1. JSON 파일 로드
with open('data\\merged_add_summary.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# 2. DataFrame 생성
df = pd.DataFrame(json_data)
df.head()

,source,title,content
0,법무부 외국인 체류관리,외국인 체류관리,text:\n법무부는 합법적인 이주를 장려하고 촉진하며 지원하기 위하여 다양한 정책...
1,법무부 외국인 체류관리,노동시장의 수요를 고려한 외국인유입정책,"text:\n법무부는 고용노동부, 산업통상자원부 등 관계부처와 유기적으로 협력하여 ..."
2,법무부 외국인 체류관리,계절근로자 제도,text:\n농‧어촌의 고질적인 인력부족 문제를 해소하기 위하여 관계부처와 협업하여...
3,법무부 외국인 체류관리,외국인 숙련기능인력 점수제 비자,text:\n주조‧금형‧용접 등 뿌리산업과 중소 제조업 등 심각한 인력난을 겪고 있...
4,법무부 외국인 체류관리,해외 우수인재 유치,text:\n국가경쟁력에 직결되는 우수인재‧전문인력의 유치를 위해 관리자‧전문직군에...


In [3]:
preprocessed_df = df.assign(content=df["content"].apply(preprocess))
preprocessed_df.head()

,source,title,content
0,법무부 외국인 체류관리,외국인 체류관리,text:\n법무부는 합법적인 이주를 장려하고 촉진하며 지원하기 위하여 다양한 정책...
1,법무부 외국인 체류관리,노동시장의 수요를 고려한 외국인유입정책,"text:\n법무부는 고용노동부, 산업통상자원부 등 관계부처와 유기적으로 협력하여 ..."
2,법무부 외국인 체류관리,계절근로자 제도,text:\n농‧어촌의 고질적인 인력부족 문제를 해소하기 위하여 관계부처와 협업하여...
3,법무부 외국인 체류관리,외국인 숙련기능인력 점수제 비자,text:\n주조‧금형‧용접 등 뿌리산업과 중소 제조업 등 심각한 인력난을 겪고 있...
4,법무부 외국인 체류관리,해외 우수인재 유치,text:\n국가경쟁력에 직결되는 우수인재‧전문인력의 유치를 위해 관리자‧전문직군에...


In [4]:
# DataFrame을 JSON으로 변환
json_data = preprocessed_df.to_dict(orient='records')
json_data[0]

{'source': '법무부 외국인 체류관리',
 'title': '외국인 체류관리',
 'content': 'text:\n법무부는 합법적인 이주를 장려하고 촉진하며 지원하기 위하여 다양한 정책적 지원방안을 시행하고 있습니다.\n2013년부터 교수/연구원 등 전문인력, 의료관광객, 단체관광객 등 일부 외국인의 경우, 재외공관을 방문하지 않고도 온라인(비자포털, www.visa.go.kr)으로 대한민국 비자를 신청/발급 받아 신속, 편리하게 입국할 수 있도록 전자비자 제도를 시행함으로써 비자신청 민원편의를 도모하고 있습니다.\n또한 유학비자를 발급받고자 하는 학생들의 경우, 우수한 학습프로그램과 유학생 지원체계를 보유하고 있는 학교를 선택하도록 장려함으로써 우수한 외국 학생들이 한국에서 인적 자본을 성장시킬 수 있도록 지속적으로 지원하고 있습니다.'}

In [5]:
import json

#JSON 파일로 저장
with open('preprocessed.json', 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=2)

#CSV로 저장
preprocessed_df.to_csv('preprocessed.csv', index=False, encoding='utf-8')